In [12]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import collections

from wordcloud import WordCloud, STOPWORDS
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster

import warnings
warnings.filterwarnings('ignore')

In [13]:
df = pd.read_csv('Crime_Reports_Austin.csv')

In [14]:
df.head()

,Incident Number,Highest Offense Description,Highest Offense Code,Family Violence,Occurred Date Time,Occurred Date,Occurred Time,Report Date Time,Report Date,Report Time,...,Y-coordinate,Latitude,Longitude,Location,Zip Codes,Single Member Council Districts,BOUNDARIES_single_member_districts,Zoning Review Cases_data,Neighborhood Planning Areas,Boundaries: City of Austin Council Districts
0,20121171927,RECKLESS DAMAGE,1401,N,04/26/2012 10:34:00 PM,04/26/2012,2234.0,04/26/2012 11:04:00 PM,04/26/2012,2304.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2006471156,FAMILY DISTURBANCE,3400,N,02/16/2006 02:25:00 PM,02/16/2006,1425.0,02/16/2006 02:25:00 PM,02/16/2006,1425.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20173300229,FAMILY DISTURBANCE,3400,N,11/26/2017 07:43:00 AM,11/26/2017,743.0,11/26/2017 07:43:00 AM,11/26/2017,743.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20045044338,TAMPERING WITH ID NUMBER,2719,N,09/14/2004 03:32:00 PM,09/14/2004,1532.0,09/14/2004 03:32:00 PM,09/14/2004,1532.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2006960811,FAMILY DISTURBANCE,3400,N,04/06/2006 10:29:00 AM,04/06/2006,1029.0,04/06/2006 10:29:00 AM,04/06/2006,1029.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2377481 entries, 0 to 2377480
Data columns (total 33 columns):
 #   Column                                        Dtype  
---  ------                                        -----  
 0   Incident Number                               int64  
 1   Highest Offense Description                   object 
 2   Highest Offense Code                          int64  
 3   Family Violence                               object 
 4   Occurred Date Time                            object 
 5   Occurred Date                                 object 
 6   Occurred Time                                 float64
 7   Report Date Time                              object 
 8   Report Date                                   object 
 9   Report Time                                   float64
 10  Location Type                                 object 
 11  Address                                       object 
 12  Zip Code                                      float64
 1

**Check for all the missing values and count them.**

In [5]:
df.isna().sum().sort_values(ascending=False)

Category Description                            1504295
UCR Category                                    1504295
Zoning Review Cases_data                        1139950
Neighborhood Planning Areas                      693396
Clearance Status                                 606136
Clearance Date                                   337704
Single Member Council Districts                   45688
BOUNDARIES_single_member_districts                43992
Boundaries: City of Austin Council Districts      43844
Council District                                  41985
Zip Codes                                         37606
Latitude                                          37525
Location                                          37525
Longitude                                         37525
Location Type                                     19007
Census Tract                                      17996
Zip Code                                           9922
PRA                                             

**Let's select only those columns that we need.**

In [6]:
column =['Incident Number', 
         'Highest Offense Code', 
         'Highest Offense Description', 
         'Occurred Date Time', 
         'Occurred Date', 
         'Address', 
         'Zip Code', 
         'Location Type', 
         'X-coordinate', 
         'Y-coordinate', 
         'Longitude', 
         'Latitude', 
         'Council District']

df1 = df.loc[:,column]
df1.head()

,Incident Number,Highest Offense Code,Highest Offense Description,Occurred Date Time,Occurred Date,Address,Zip Code,Location Type,X-coordinate,Y-coordinate,Longitude,Latitude,Council District
0,20121171927,1401,RECKLESS DAMAGE,04/26/2012 10:34:00 PM,04/26/2012,3154 HWY 71 E,NaN,RESIDENCE / HOME,NaN,NaN,NaN,NaN,NaN
1,2006471156,3400,FAMILY DISTURBANCE,02/16/2006 02:25:00 PM,02/16/2006,7000 DECKER 1422,NaN,RESIDENCE / HOME,NaN,NaN,NaN,NaN,NaN
2,20173300229,3400,FAMILY DISTURBANCE,11/26/2017 07:43:00 AM,11/26/2017,13204 LIPTON LP,NaN,RESIDENCE / HOME,NaN,NaN,NaN,NaN,NaN
3,20045044338,2719,TAMPERING WITH ID NUMBER,09/14/2004 03:32:00 PM,09/14/2004,3301 CR 100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2006960811,3400,FAMILY DISTURBANCE,04/06/2006 10:29:00 AM,04/06/2006,5005 W FRANCES PL,NaN,RESIDENCE / HOME,NaN,NaN,NaN,NaN,NaN


In [7]:
df1.isna().sum().sort_values(ascending=False)

Council District               41985
Longitude                      37525
Latitude                       37525
Location Type                  19007
Zip Code                        9922
X-coordinate                    5969
Y-coordinate                    5962
Occurred Date Time               137
Address                           13
Incident Number                    0
Highest Offense Code               0
Highest Offense Description        0
Occurred Date                      0
dtype: int64

In [8]:
df1.shape

(2377481, 13)

Compared to the 2M rows, the missing rows seem insignificant.
We'll drop the rows with missing data.

In [9]:
df1.dropna(inplace=True)

In [10]:
df1.shape
print(round(2287541/2377481 *100,2))

96.22


We have retained around 96% data even after dropping rows with missing values.

Drop all the duplicated entries based on 'Incident Number'

In [11]:
df1.drop_duplicates(subset=['Incident Number'], inplace=True)

For the date comparison we need to change dtype of columns Occurred Date Time, Occurred Date from object to datetime.
Similarly, we are changing the data type of Zip Code and Council District into int.

In [ ]:
df1['Zip Code'] =df1['Zip Code'].astype(int)
df1['Council District'] =df1['Council District'].astype(int)
df1['Occurred Date'] = pd.to_datetime(df1['Occurred Date'])  
df1['Occurred Date Time'] = pd.to_datetime(df1['Occurred Date Time']) 
# Y['Occurred Date'] = Y['Occurred Date'].dt.strftime('%m/%d/%Y')

Let's convert the uppercased data to camel-cased.

In [ ]:
df1['Highest Offense Description'] = df1['Highest Offense Description'].str.title()
df1['Location Type'] = df1['Location Type'].str.title() 
df1['Address'] = df1['Address'].str.title()

Now, extracting the month, year, week and day of month from the date.

In [ ]:
df1['Month'] = df1['Occurred Date Time'].dt.strftime('%B')
df1['Year'] = df1['Occurred Date Time'].dt.strftime('%Y')
df1['Week'] = df1['Occurred Date Time'].dt.day_name()
df1["DayOfMonth"] = df1['Occurred Date Time'].dt.day

In [ ]:
df1.sample(5)

Replacing same entries with different description by common description. 

In [ ]:
mapping = {'Agg Assault':'Aggravated Assault',
           'Agg Assault Fam/Date Violence':'Aggravated Assault',
           'Burg Of Res - Fam/Dating Aslt':'Burglary Of Residence',
           'Att Burglary Of Residence':'Burglary Of Residence',
           'Theft-No Suspect/Followup':'Theft',
           'Burglary Of Veh-No Suspect/Fu':'Burglary Of Vehicle',
           'Assault W/Injury-Fam/Date Viol':'Assault With Injury',
           'Assault By Threat Fam/Dating': 'Assault By Threat',
           'Assault By Contact Fam/Dating':'Assault By Contact',
           'Forgery By Alteration':'Forgery And Passing',
           'Cred Card Abuse - Other': 'Debit/Credit Card Abuse',
           'Debit Card Abuse': 'Debit/Credit Card Abuse',
           'Criminal Mischief-No Suspect':'Criminal Mischief',
           'Poss Controlled Sub/Narcotic': 'Narcotic Possession',
           'Poss Controlled Sub/Syn Narc':'Narcotic Possession',
           'Poss Controlled Sub/Other':'Narcotic Possession',
           'Del Controlled Sub/Narcotic':'Narcotic Possession',
           'Del Controlled Sub/Syn Narc':'Narcotic Possession',
           'Del Controlled Sub/Other':'Narcotic Possession',
           'Dwi': 'DWI',
           'Dwi 2Nd': 'DWI',
           'Criminal Trespass/Transient':'Criminal Trespass',
           'Criminal Trespass/Hotel':'Criminal Trespass',
           'Terroristic Threat-Fam/Dat Vio':'Terroristic Threat',
           'Agg Robbery/Deadly Weapon': 'Aggravated Robbery',
           'Robbery By Assault': 'Aggravated Robbery'
          }


df1['Highest Offense Description'] = df1['Highest Offense Description'].replace(mapping)